# Interactive Quick Start with StagecoachML

This notebook provides an interactive introduction to StagecoachML running in your browser with Pyodide!

**How to use this notebook:**
- Run cells by clicking them and pressing **Shift + Enter**
- Modify code and re-run cells to experiment
- Everything runs locally in your browser - no server required!

## Installation & Setup

This notebook runs Python via **Pyodide** (Python compiled to WebAssembly) directly in your browser. 

Let's install StagecoachML and its dependencies using `micropip` (Pyodide's package installer):

In [ ]:
# Import all required libraries first
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split

from stagecoachml import StagecoachClassifier, StagecoachRegressor

# Install StagecoachML in the browser using micropip
print("Installing StagecoachML and dependencies...")

try:
    import micropip
    # Install StagecoachML from PyPI
    await micropip.install("stagecoachml", deps=True)
    print("✅ StagecoachML installed successfully!")
except ImportError:
    # Fallback for non-Pyodide environments (local development)
    print("ℹ️  Running outside Pyodide - assuming StagecoachML is already available")

print("🎉 All libraries loaded successfully! Ready to go!")

## 🚀 Let's Get Started!

**Run the cell above first** (click it and press Shift+Enter) to install StagecoachML.

Once installation completes, you can run all the examples below. Feel free to:
- 📝 **Modify the code** and re-run cells to experiment
- 🔄 **Change parameters** to see different results  
- 📊 **Try different datasets** or algorithms

Let's start with some real examples!

## Regression Example

Let's build a two-stage regression model with the diabetes dataset:

In [ ]:
# Load the diabetes dataset
diabetes = load_diabetes(as_frame=True)
X = diabetes.frame.drop(columns=["target"])
y = diabetes.frame["target"]

# Split features into "early" and "late"
features = list(X.columns)
mid = len(features) // 2
early_features = features[:mid]  # First half arrives early
late_features = features[mid:]  # Second half arrives later

print(f"Dataset: {len(X)} samples, {len(features)} features")
print(f"Early features ({len(early_features)}): {early_features}")
print(f"Late features ({len(late_features)}): {late_features}")

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the two-stage model
model = StagecoachRegressor(
    stage1_estimator=LinearRegression(),
    stage2_estimator=RandomForestRegressor(n_estimators=50, random_state=42),
    early_features=early_features,
    late_features=late_features,
    residual=True,
    use_stage1_pred_as_feature=True,
)

# Train the model
model.fit(X_train, y_train)
print("Model trained successfully!")

In [ ]:
# Evaluate the model
stage1_pred = model.predict_stage1(X_test)
final_pred = model.predict(X_test)

stage1_r2 = r2_score(y_test, stage1_pred)
final_r2 = r2_score(y_test, final_pred)

print(f"Stage-1 R² (early features only): {stage1_r2:.3f}")
print(f"Final R² (all features): {final_r2:.3f}")
print(f"\nImprovement from stage-2: {final_r2 - stage1_r2:.3f}")

## Classification Example

Now let's try a classification task with breast cancer data:

In [ ]:
# Load breast cancer dataset
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

# Split features
features = list(X.columns)
mid = len(features) // 2
early = features[:mid]
late = features[mid:]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Create two-stage classifier
clf = StagecoachClassifier(
    stage1_estimator=LogisticRegression(max_iter=1000, random_state=42),
    stage2_estimator=RandomForestClassifier(n_estimators=50, random_state=42),
    early_features=early,
    late_features=late,
    use_stage1_pred_as_feature=True,
)

# Train
clf.fit(X_train, y_train)
print("Classifier trained!")

In [ ]:
# Evaluate
stage1_proba = clf.predict_stage1_proba(X_test)
stage1_pred = (stage1_proba >= 0.5).astype(int)
final_pred = clf.predict(X_test)

stage1_acc = accuracy_score(y_test, stage1_pred)
final_acc = accuracy_score(y_test, final_pred)

print(f"Stage-1 Accuracy (early features): {stage1_acc:.3f}")
print(f"Final Accuracy (all features): {final_acc:.3f}")
print(f"\nImprovement from stage-2: {final_acc - stage1_acc:.3f}")

## Interactive Exploration

Try modifying the code above to:
- Change the split of early vs late features
- Try different estimators (e.g., `GradientBoostingRegressor`, `SVC`)
- Adjust hyperparameters
- Set `residual=False` to see the difference

This is the power of interactive documentation - you can experiment right here!